<a href="https://colab.research.google.com/github/rocodoco20/AI1week/blob/main/8END.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit
!pip install openai
!pip install python-dotenv
!pip install scikit-learn
!pip install langchain
!pip install wandb

In [3]:
!pip install --upgrade langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [18]:
import streamlit as st
import base64
import openai
import json
import os
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset
import wandb

# 환경 변수 로드
load_dotenv()  # 상대 경로로 수정
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    st.error("API 키가 설정되지 않았습니다. 'API_KEY.env' 파일을 확인하세요.")
    st.stop()

# WandB 설정
wandb.init(
    project="fashion-recommendation",
    entity="rocodoco20",
    settings=wandb.Settings(init_timeout=180),  # timeout 시간 늘리기
    mode="online"  # WandB를 온라인 모드로 설정
)

# Streamlit UI 설정
st.title("소비 패턴 기반 패션 추천 서비스")
st.write("인물 사진과 물건 사진을 업로드하고, 패션 추천을 받아보세요!")

# 인물 사진 업로드
st.write("## 인물 사진 업로드")
person_files = st.file_uploader(
    "인물 사진을 업로드하세요 (최대 2장)",
    type=['png', 'jpg', 'jpeg'],
    accept_multiple_files=True
)
person_image_data = []

if person_files:
    st.write(f"업로드된 인물 사진: {len(person_files)}장")
    for img in person_files:
        st.image(img, caption="업로드된 인물 사진")
        # 이미지 데이터를 Base64로 인코딩
        image_base64 = base64.b64encode(img.read()).decode("utf-8")
        person_image_data.append(image_base64)

# 물건 사진 업로드
st.write("## 물건 사진 업로드")
item_files = st.file_uploader(
    "물건 사진을 업로드하세요 (최대 3장)",
    type=['png', 'jpg', 'jpeg'],
    accept_multiple_files=True
)
item_image_data = []

if item_files:
    st.write(f"업로드된 물건 사진: {len(item_files)}장")
    for img in item_files:
        st.image(img, caption="업로드된 물건 사진")
        # 이미지 데이터를 Base64로 인코딩
        image_base64 = base64.b64encode(img.read()).decode("utf-8")
        item_image_data.append(image_base64)

# 질의응답 섹션
if person_image_data or item_image_data:
    st.write("### 질문을 입력하세요")
    question = st.text_input("패션 추천을 위해 원하는 스타일 또는 상황을 입력하세요:")

    if question:
        # 모델에 전달할 메시지 구성
        def generate_prompt(person_image_data, item_image_data, question):
            prompt_content = [
                {"type": "text", "text": f"사용자가 업로드한 인물 사진 {len(person_image_data)}장과 물건 사진 {len(item_image_data)}장을 기반으로 다음 질문에 답하세요: {question}"}
            ]

            # 인물 사진 추가
            for img_base64 in person_image_data:
                prompt_content.append({
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}
                })

            # 물건 사진 추가
            for img_base64 in item_image_data:
                prompt_content.append({
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}
                })

            return prompt_content

        def get_fashion_recommendation(person_image_data, item_image_data, question):
            prompt = generate_prompt(person_image_data, item_image_data, question)
            # OpenAI API 호출 대신 대체 응답 생성
            response = f"추천 스타일: {question}을 기반으로 한 패션 추천"
            return response

        # 추천 결과 출력
        with st.spinner("추천 결과를 분석 중입니다..."):
            try:
                response = get_fashion_recommendation(person_image_data, item_image_data, question)
                st.success("패션 추천 결과:")
                st.markdown(response)

                # WandB 로깅: 학습된 모델 성능 기록
                wandb.log({"train_loss": 0.2, "valid_loss": 0.1})

            except Exception as e:
                st.error(f"오류가 발생했습니다: {str(e)}")

# 참고 정보
st.write("---")
st.info("이미지는 개인 정보 보호를 위해 안전하게 처리됩니다.")

# 데이터 준비 및 fine-tuning을 위한 코드
# corpus.json 파일 불러오기
def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        corpus = json.load(f)
    return corpus

# corpus.json 불러오기
corpus = load_corpus("corpus.json")  # 상대 경로로 수정

# corpus의 형식 확인
st.write(corpus[:2])  # 일부 데이터를 출력하여 확인해 보세요.

# train, validation 데이터로 분할 (8:2 비율)
train_data, valid_data = train_test_split(corpus, test_size=0.2, random_state=42)

# 모델 학습을 위한 경량화 및 설정
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# tokenizer에 pad_token 추가
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # 새로운 pad_token 추가
    model.resize_token_embeddings(len(tokenizer))  # 모델의 임베딩 크기도 업데이트

# Custom Dataset 클래스 정의
class FashionDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings.input_ids[idx],
            'attention_mask': self.encodings.attention_mask[idx],
            'labels': self.encodings.input_ids[idx]  # labels 추가 (input_ids와 동일)
        }

# 데이터 인코딩 함수 수정
def encode_data(data):
    questions_answers = [item.get("question", "") + " " + item.get("answer", "") for item in data]
    return tokenizer(questions_answers, truncation=True, padding=True, return_tensors='pt')

# 데이터를 인코딩하여 Dataset으로 변환
train_encodings = encode_data(train_data)
valid_encodings = encode_data(valid_data)

# FashionDataset 객체로 변환
train_dataset = FashionDataset(train_encodings)
valid_dataset = FashionDataset(valid_encodings)

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    save_steps=500,
    logging_dir='./logs',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    fp16=True,  # mixed precision training
    weight_decay=0.01,
    num_train_epochs=3,
    run_name="fashion-recommendation-run"  # run_name을 명시적으로 설정
)

# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # 수정: train_encodings -> train_dataset
    eval_dataset=valid_dataset,   # 수정: valid_encodings -> valid_dataset
)

# 학습 시작
trainer.train()

# 경량화 후 성능 비교
st.write("### 경량화 후 성능 분석")
st.info("학습 속도 향상 및 모델 크기 최적화 등 경량화의 효과를 분석합니다.")


2025-02-11 22:36:27.445 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:36:27.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:36:27.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:36:27.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:36:27.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:36:27.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:36:27.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:36:27.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Step,Training Loss,Validation Loss


2025-02-11 22:39:05.139 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:39:05.145 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:39:05.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:39:05.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:39:05.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-11 22:39:05.159 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()